<a href="https://colab.research.google.com/github/fMurugi/complete-neural-network-course/blob/main/Feed_Forward_Neural_Network_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [22]:
input_size = 784
hidden_size =400
out_size = 10
epochs = 10
batch_size = 100
learning_rate = 0.001


In [23]:
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

In [24]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                    batch_size=batch_size,
                                    shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [25]:
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,out_size):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size) #firsst layer
        self.relu = nn.ReLU() #first layer activation
        self.fc2 = nn.Linear(hidden_size,hidden_size) #second layer
        self.fc3 = nn.Linear(hidden_size,out_size) #second layer activation

  def forward(self,x): #x is data the images turned to tensor
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out


In [26]:
# create an object of the class, which represents our network
net = Net(input_size,hidden_size,out_size)
CUDA = torch.cuda.is_available()
CUDA
if CUDA:
  net = net.cuda()
# The loss function.The Cross Entropy loss comes along with softmax.Therefore no need to specify softmax
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [28]:
#train the network
correct_train =0
total_train = 0
for epoch in range(epochs):
  for i, (images,labels) in enumerate(train_loader):
      images = Variable(images.view(-1,28*28))
      labels = Variable(labels)
      if CUDA:
        images = images.cuda()
        labels = labels.cuda()
      #clear gradient to avoid accumulation when updatin weight
      optimizer.zero_grad()
      outputs = net(images)
      _,predicted = torch.max(outputs.data,1)
      total_train += labels.size(0)
      if CUDA:
        correct_train += (predicted.cpu() == labels.cpu()).sum()
      else:
        correct_train += (predicted == labels).sum()
      loss = criterion(outputs,labels)
      loss.backward()
      optimizer.step()
      if (i+1)%100 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f} Training Accuracy:{}:.3f%'.format(
            epoch+1,epochs,i+1,len(train_dataset)//batch_size,loss.item(),(100*correct_train/total_train))
        )
print("DONE TRAINING")




Epoch [1/10], Step [100/600], Loss: 0.2808 Training Accuracy:83.58000183105469%
Epoch [1/10], Step [200/600], Loss: 0.1810 Training Accuracy:87.93000030517578%
Epoch [1/10], Step [300/600], Loss: 0.2557 Training Accuracy:89.76000213623047%
Epoch [1/10], Step [400/600], Loss: 0.0956 Training Accuracy:90.97000122070312%
Epoch [1/10], Step [500/600], Loss: 0.1553 Training Accuracy:91.89600372314453%
Epoch [1/10], Step [600/600], Loss: 0.0573 Training Accuracy:92.49500274658203%
DONE TRAINING
Epoch [2/10], Step [100/600], Loss: 0.0855 Training Accuracy:93.105712890625%
Epoch [2/10], Step [200/600], Loss: 0.1229 Training Accuracy:93.55374908447266%
Epoch [2/10], Step [300/600], Loss: 0.0291 Training Accuracy:93.96333312988281%
Epoch [2/10], Step [400/600], Loss: 0.1732 Training Accuracy:94.30799865722656%
Epoch [2/10], Step [500/600], Loss: 0.0715 Training Accuracy:94.5654525756836%
Epoch [2/10], Step [600/600], Loss: 0.0606 Training Accuracy:94.78250122070312%
DONE TRAINING
Epoch [3/10], S

In [29]:
#test the network (No loss calculation and no weight update)
correct =0
total = 0
for images,labels in test_loader:
  images = Variable(images.view(-1,28*28))
  if CUDA:
    images = images.cuda()
  outputs = net(images)
  _,predicted = torch.max(outputs.data,1)
  total += labels.size(0)
  if CUDA:
    correct += (predicted.cpu() == labels.cpu()).sum()
  else:
    correct += (predicted == labels).sum()
print('Accuracy of the network: {} %'.format(100 * correct / total))


Accuracy of the network: 97.88999938964844 %
